In [1]:
from tensorflow.keras import datasets
import tensorflow as tf
import optuna

# loading CIFAR100 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()
def train_augment(image, label):
    # standarize
    image = tf.image.per_image_standardization(image)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    image = tf.image.random_crop(image, size=[32, 32, 3])
    
    return image, label  # Return both image and label

def test_augment(image, label):
    return image, label

In [2]:
from sklearn.model_selection import train_test_split
# splitting validation set and test set
test_images, val_images, test_labels, val_labels = train_test_split(test_images, test_labels, test_size=0.5, random_state=42)

In [3]:
train_ds = (
    tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    .map(train_augment, num_parallel_calls=tf.data.AUTOTUNE)  # Now works correctly
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)
val_ds = (
    tf.data.Dataset.from_tensor_slices((val_images, val_labels))
    .map(test_augment, num_parallel_calls=tf.data.AUTOTUNE)  # Ensure label is passed correctly
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((test_images, test_labels))
    .map(test_augment, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)


Finding best architecture complexity between 1, 2 and 3 conv layers

In [4]:
def conv_block(k, filter_size = (3, 3), dropout=0, regularization=0):
    block = [
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(k, filter_size, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularization)),
    ]
    if dropout:
        block.append(tf.keras.layers.Dropout(dropout))
    return block

def final_block(dropout = 0, regularization=0):
    block = [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularization)),
    ]
    if dropout:
        block.append(tf.keras.layers.Dropout(dropout))
    block.append(tf.keras.layers.Dense(100, activation='softmax'))
    return block


In [5]:
def create_model(num_layers,base = 5, kernels = (3, 3), regularization=0, dropout=0):
    inputs = tf.keras.layers.Input(shape=(32, 32, 3))
    x = tf.keras.layers.Conv2D(2**base, kernels, activation='relu')(inputs)
    
    for layer_number in range(num_layers):
        for sub_layer in conv_block(2**(base+layer_number), kernels, dropout, regularization):
            try :
                x = sub_layer(x)
            except ValueError:
                pass
            
    for sub_layer in final_block():
        x = sub_layer(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)
        
    return model

In [6]:
model = create_model(3, kernels = (7,7), regularization=1, dropout=1, droplayer='conv')

In [7]:
def objective(trial):
    lr = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    model = create_model(num_layers = trial.suggest_int('num_layers', 0, 3), 
                         base = trial.suggest_int('base', 4, 8),
                         kernels = trial.suggest_categorical("kernel_size", [(3, 3), (5, 5), (7, 7)]),
                         regularization = trial.suggest_float('regularization', 0, 1e-3),
                         dropout = trial.suggest_float('dropout', 0, 0.3),
                         )
                         
                    

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_ds, epochs=10, validation_data=(val_ds), verbose=0)
    return model.evaluate(test_ds)[1]

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Run Optuna optimization
study = optuna.create_study(direction="maximize")  
study.optimize(objective, n_trials=100)
# Best hyperparameters
print("Best hyperparameters:", study.best_params)

[I 2025-02-10 23:22:14,116] A new study created in memory with name: no-name-19ce0ebd-0891-4e13-9139-c745fa3002d6


157/157 [==============================] - 1s 4ms/step - loss: 4.6039 - accuracy: 0.0104


[I 2025-02-10 23:25:08,255] Trial 0 finished with value: 0.010400000028312206 and parameters: {'learning_rate': 0.001744585696079045, 'num_layers': 0, 'base': 8, 'kernel_size': (5, 5), 'regularization': 0.00033265609780656923, 'dropout': 0.16839246773270714, 'droplayer': 'conv'}. Best is trial 0 with value: 0.010400000028312206.


157/157 [==============================] - 1s 4ms/step - loss: 4.6059 - accuracy: 0.0090


[I 2025-02-10 23:27:31,673] Trial 1 finished with value: 0.008999999612569809 and parameters: {'learning_rate': 0.0020230109517695833, 'num_layers': 3, 'base': 7, 'kernel_size': (7, 7), 'regularization': 0.0005657088143989586, 'dropout': 0.12200412943936634, 'droplayer': 'dense'}. Best is trial 0 with value: 0.010400000028312206.


...

157/157 [==============================] - 1s 6ms/step - loss: 4.6073 - accuracy: 0.0096


[I 2025-02-11 06:57:11,575] Trial 98 finished with value: 0.009600000455975533 and parameters: {'learning_rate': 5.252775438998015e-05, 'num_layers': 1, 'base': 8, 'kernel_size': (5, 5), 'regularization': 0.0004108040757035723, 'dropout': 0.07452641859008904, 'droplayer': 'conv'}. Best is trial 74 with value: 0.33719998598098755.


157/157 [==============================] - 1s 7ms/step - loss: 3.6012 - accuracy: 0.1768


[I 2025-02-11 07:03:53,359] Trial 99 finished with value: 0.17679999768733978 and parameters: {'learning_rate': 3.3024929863761084e-05, 'num_layers': 3, 'base': 8, 'kernel_size': (7, 7), 'regularization': 0.00036524153533334025, 'dropout': 0.11001793295981947, 'droplayer': 'conv'}. Best is trial 74 with value: 0.33719998598098755.


Best hyperparameters: {'learning_rate': 8.423513376164038e-05, 'num_layers': 3, 'base': 8, 'kernel_size': (5, 5), 'regularization': 0.0005147656842312698, 'dropout': 0.12774143796296356, 'droplayer': 'conv'}


Optuna optimizes hyperparameters using search algorithms, such as:

1️⃣ Bayesian Optimization (TPE - Tree-structured Parzen Estimator)
- Learns from past trials to suggest better hyperparameters.
- Efficient for complex search spaces.

2️⃣ Grid Search & Random Search
- Random search: Tries random values in the defined range.
- Grid search: Tests all possible combinations (not recommended for large search spaces).

3️⃣ Pruning (Early Stopping)
- Stops bad trials early to save compute time.

In [10]:
import pickle

# Save the model
with open('best_params.pkl', 'wb') as f:
    pickle.dump(study.best_params, f)

In [13]:
import pandas as pd
pd.DataFrame(study.trials_dataframe()).to_csv('study.csv', index=False)

In [16]:
pd.DataFrame(study.best_params).iloc[:1].T

,0
learning_rate,0.000084
num_layers,3
base,8
kernel_size,5
regularization,0.000515
dropout,0.127741
droplayer,conv


In [21]:
best_params = study.best_params.copy()

model = create_model(num_layers=best_params['num_layers'],
                     base=best_params['base'],
                     kernels=best_params['kernel_size'],
                     regularization=best_params['regularization'],
                     dropout=best_params['dropout'],)

In [22]:
# WANDB
import wandb
from wandb.integration.keras import WandbMetricsLogger

# Start a new W&B run
wandb.init(project="cifar100")

# Early stopping for f1
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)



wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ahmad\_netrc
wandb: Currently logged in as: mlahmadmostafa (mlahmadmostafa-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [26]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(tf.clip_by_value(y_pred, 0, 1))
        y_true = tf.cast(y_true, tf.float32)

        tp = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32))
        fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, tf.float32))
        fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), tf.float32))

        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.true_positives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)


In [28]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy', F1Score()])

model.fit(train_ds, epochs=100, validation_data=(val_ds), verbose=2, 
          callbacks=[WandbMetricsLogger(), early_stopping])

Epoch 1/100
1563/1563 - 41s - loss: 3.7126 - accuracy: 0.1588 - f1_score: 0.0011 - val_loss: 3.5382 - val_accuracy: 0.1814 - val_f1_score: 0.0011 - 41s/epoch - 26ms/step
Epoch 2/100
1563/1563 - 40s - loss: 3.4663 - accuracy: 0.1990 - f1_score: 0.0016 - val_loss: 3.3348 - val_accuracy: 0.2210 - val_f1_score: 0.0013 - 40s/epoch - 25ms/step
Epoch 3/100
1563/1563 - 39s - loss: 3.2908 - accuracy: 0.2290 - f1_score: 0.0021 - val_loss: 3.2080 - val_accuracy: 0.2374 - val_f1_score: 0.0019 - 39s/epoch - 25ms/step
Epoch 4/100
1563/1563 - 40s - loss: 3.1551 - accuracy: 0.2519 - f1_score: 0.0026 - val_loss: 3.0930 - val_accuracy: 0.2596 - val_f1_score: 0.0022 - 40s/epoch - 25ms/step
Epoch 5/100
1563/1563 - 40s - loss: 3.0417 - accuracy: 0.2726 - f1_score: 0.0031 - val_loss: 3.0706 - val_accuracy: 0.2768 - val_f1_score: 0.0026 - 40s/epoch - 25ms/step
Epoch 6/100
1563/1563 - 39s - loss: 2.9465 - accuracy: 0.2960 - f1_score: 0.0035 - val_loss: 2.9360 - val_accuracy: 0.2952 - val_f1_score: 0.0031 - 39

KeyboardInterrupt: 